# LangChain Agents and Tools Tutorial 🤖

This notebook provides a comprehensive guide to building and using LangChain agents with various tools.

## Learning Objectives
- Understand agent types and architectures
- Build custom tools for specific tasks
- Create multi-tool agents
- Handle tool errors and edge cases
- Optimize agent performance

## Setup and Imports

In [ ]:
# Install required packages (run if needed)
# !pip install langchain openai python-dotenv

import os
import sys
from pathlib import Path

# Add utils to path
sys.path.append(str(Path.cwd().parent / 'utils'))

from langchain.agents import initialize_agent, AgentType, Tool
from langchain.chat_models import ChatOpenAI
from langchain.tools import BaseTool
from langchain.schema import AgentAction, AgentFinish
from langchain.memory import ConversationBufferMemory

# Import our utilities
try:
    from utils.config import get_api_key
    api_key = get_api_key('openai')
    if not api_key:
        print("⚠️ No API key found. Some examples will use mock responses.")
        DEMO_MODE = True
    else:
        DEMO_MODE = False
        print("✅ API key loaded successfully")
except ImportError:
    print("⚠️ Utils not available. Using environment variables.")
    api_key = os.getenv('OPENAI_API_KEY')
    DEMO_MODE = not api_key

print(f"Demo mode: {DEMO_MODE}")

## 1. Basic Agent Setup

Let's start with a simple agent that can use basic tools.

In [ ]:
# Initialize LLM
if not DEMO_MODE:
    llm = ChatOpenAI(openai_api_key=api_key, temperature=0)
else:
    # Mock LLM for demo
    class MockLLM:
        def predict(self, text):
            return "This is a mock response since no API key is available."
    llm = MockLLM()

print("LLM initialized")

## 2. Creating Custom Tools

Tools are the building blocks that give agents capabilities.

In [ ]:
# Simple calculator tool
def calculator(expression: str) -> str:
    """Safely evaluate mathematical expressions."""
    try:
        # Safe evaluation - only allow basic operations
        allowed_chars = set('0123456789+-*/.() ')
        if not set(expression).issubset(allowed_chars):
            return "Error: Only basic mathematical operations are allowed"
        
        result = eval(expression)
        return f"The result is: {result}"
    except Exception as e:
        return f"Error calculating: {str(e)}"

# Text analysis tool
def text_analyzer(text: str) -> str:
    """Analyze text properties like length, word count, etc."""
    try:
        words = text.split()
        chars = len(text)
        sentences = text.count('.') + text.count('!') + text.count('?')
        
        analysis = {
            'characters': chars,
            'words': len(words),
            'sentences': sentences,
            'average_word_length': round(sum(len(word) for word in words) / len(words), 2) if words else 0
        }
        
        return f"Text analysis: {analysis}"
    except Exception as e:
        return f"Error analyzing text: {str(e)}"

# Create tool objects
tools = [
    Tool(
        name="Calculator",
        func=calculator,
        description="Useful for mathematical calculations. Input should be a mathematical expression."
    ),
    Tool(
        name="TextAnalyzer",
        func=text_analyzer,
        description="Analyze text properties like word count, character count, and sentence count."
    )
]

print(f"Created {len(tools)} tools:")
for tool in tools:
    print(f"- {tool.name}: {tool.description}")

## 3. Creating an Agent

Now let's create an agent that can use these tools.

In [ ]:
# Create memory for the agent
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

if not DEMO_MODE:
    # Initialize agent with tools
    agent = initialize_agent(
        tools,
        llm,
        agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        memory=memory,
        verbose=True,
        handle_parsing_errors=True
    )
    
    print("✅ Agent created successfully!")
else:
    print("⚠️ Agent creation skipped in demo mode")
    agent = None

## 4. Testing the Agent

Let's test our agent with various tasks.

In [ ]:
# Test the agent with different types of queries
test_queries = [
    "What is 25 * 4 + 10?",
    "Analyze this text: 'Hello world! This is a test sentence. How are you?'",
    "Calculate the area of a circle with radius 5 (use 3.14159 for pi)",
    "How many words are in the phrase 'LangChain is amazing for building AI applications'?"
]

if agent:
    print("🧪 Testing agent with various queries...\n")
    
    for i, query in enumerate(test_queries, 1):
        print(f"Query {i}: {query}")
        try:
            response = agent.run(query)
            print(f"Response: {response}\n")
        except Exception as e:
            print(f"Error: {str(e)}\n")
else:
    # Demo mode - show what the responses might look like
    print("🧪 Demo responses (API key required for actual agent execution):\n")
    
    demo_responses = [
        "I'll calculate 25 * 4 + 10. Using the calculator tool: The result is 110.",
        "Let me analyze that text. Using text analyzer: {'characters': 43, 'words': 8, 'sentences': 2, 'average_word_length': 4.0}",
        "I'll calculate the area using A = π * r². Using calculator: 3.14159 * 5 * 5 = 78.54",
        "I'll count the words in that phrase. Using text analyzer: The phrase contains 8 words."
    ]
    
    for i, (query, response) in enumerate(zip(test_queries, demo_responses), 1):
        print(f"Query {i}: {query}")
        print(f"Response: {response}\n")

## 5. Advanced Tool Creation

Let's create more sophisticated tools using the BaseTool class.

In [ ]:
from typing import Optional, Type
from pydantic import BaseModel, Field

class FileManagerInput(BaseModel):
    """Input for file manager tool."""
    operation: str = Field(description="Operation to perform: 'create', 'read', 'list'")
    filename: Optional[str] = Field(description="Name of file to operate on")
    content: Optional[str] = Field(description="Content to write to file")

class FileManagerTool(BaseTool):
    """Tool for basic file operations."""
    name = "FileManager"
    description = "Manage files: create, read, or list files in the current directory"
    args_schema: Type[BaseModel] = FileManagerInput
    
    def _run(self, operation: str, filename: str = None, content: str = None) -> str:
        try:
            if operation == "list":
                files = [f for f in os.listdir('.') if os.path.isfile(f)]
                return f"Files in current directory: {files}"
            
            elif operation == "create" and filename and content:
                with open(filename, 'w') as f:
                    f.write(content)
                return f"File '{filename}' created successfully"
            
            elif operation == "read" and filename:
                if os.path.exists(filename):
                    with open(filename, 'r') as f:
                        content = f.read()
                    return f"Content of '{filename}': {content[:200]}{'...' if len(content) > 200 else ''}"
                else:
                    return f"File '{filename}' not found"
            
            else:
                return "Invalid operation or missing parameters"
                
        except Exception as e:
            return f"Error: {str(e)}"
    
    async def _arun(self, operation: str, filename: str = None, content: str = None) -> str:
        """Async version of the tool."""
        return self._run(operation, filename, content)

# Create advanced tools list
advanced_tools = tools + [FileManagerTool()]

print(f"Created {len(advanced_tools)} tools including advanced FileManager")

## 6. Agent with Advanced Tools

Let's create a more capable agent with our advanced tools.

In [ ]:
if not DEMO_MODE:
    # Create advanced agent
    advanced_agent = initialize_agent(
        advanced_tools,
        llm,
        agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
        memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True),
        verbose=True,
        handle_parsing_errors=True
    )
    
    print("✅ Advanced agent created!")
    
    # Test advanced functionality
    advanced_queries = [
        "Create a file called 'test.txt' with the content 'Hello from LangChain agent!'",
        "List all files in the current directory",
        "Read the content of 'test.txt' and then count how many words it contains"
    ]
    
    print("\n🚀 Testing advanced agent capabilities...\n")
    
    for i, query in enumerate(advanced_queries, 1):
        print(f"Advanced Query {i}: {query}")
        try:
            response = advanced_agent.run(query)
            print(f"Response: {response}\n")
        except Exception as e:
            print(f"Error: {str(e)}\n")
            
else:
    print("⚠️ Advanced agent testing skipped in demo mode")
    print("Demo would show file operations, content analysis, and tool chaining.")

## 7. Error Handling and Safety

Proper error handling is crucial for robust agent applications.

In [ ]:
class SafeCalculatorTool(BaseTool):
    """Calculator with enhanced safety and error handling."""
    name = "SafeCalculator"
    description = "Safely calculate mathematical expressions with error handling"
    
    def _run(self, expression: str) -> str:
        # Input validation
        if not expression or not expression.strip():
            return "Error: Empty expression provided"
        
        # Security check - only allow safe characters
        allowed_chars = set('0123456789+-*/.() ')
        if not set(expression).issubset(allowed_chars):
            return "Error: Expression contains forbidden characters. Only numbers and basic operators (+, -, *, /, parentheses) are allowed."
        
        # Length check
        if len(expression) > 100:
            return "Error: Expression too long (max 100 characters)"
        
        try:
            # Safe evaluation
            result = eval(expression)
            
            # Check for reasonable results
            if abs(result) > 1e10:
                return f"Warning: Very large result: {result:.2e}"
            
            return f"Result: {result}"
            
        except ZeroDivisionError:
            return "Error: Division by zero"
        except SyntaxError:
            return "Error: Invalid mathematical expression"
        except Exception as e:
            return f"Error: {str(e)}"
    
    async def _arun(self, expression: str) -> str:
        return self._run(expression)

# Test the safe calculator
safe_calc = SafeCalculatorTool()

test_expressions = [
    "2 + 2",          # Valid
    "10 / 0",         # Division by zero
    "2 +",            # Syntax error
    "import os",      # Security violation
    "",               # Empty input
    "1" * 200         # Too long
]

print("🛡️ Testing Safe Calculator with various inputs:\n")

for expr in test_expressions:
    result = safe_calc._run(expr)
    print(f"Input: '{expr[:20]}{'...' if len(expr) > 20 else ''}'")
    print(f"Output: {result}\n")

## 8. Agent Performance Optimization

Tips for optimizing agent performance and reliability.

In [ ]:
# Performance monitoring decorator
import time
from functools import wraps

def monitor_performance(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start_time = time.time()
        try:
            result = func(*args, **kwargs)
            success = True
            error = None
        except Exception as e:
            result = f"Error: {str(e)}"
            success = False
            error = str(e)
        
        end_time = time.time()
        execution_time = end_time - start_time
        
        print(f"Tool executed in {execution_time:.3f} seconds, Success: {success}")
        if not success:
            print(f"Error details: {error}")
        
        return result
    return wrapper

class MonitoredCalculator(BaseTool):
    """Calculator with performance monitoring."""
    name = "MonitoredCalculator"
    description = "Calculator with performance monitoring"
    
    @monitor_performance
    def _run(self, expression: str) -> str:
        # Simulate some processing time
        time.sleep(0.1)
        
        try:
            result = eval(expression)
            return f"Result: {result}"
        except Exception as e:
            return f"Error: {str(e)}"
    
    async def _arun(self, expression: str) -> str:
        return self._run(expression)

# Test monitored tool
monitored_calc = MonitoredCalculator()

print("📊 Testing performance monitoring:\n")
test_calcs = ["5 + 3", "10 * 2", "invalid_expr"]

for calc in test_calcs:
    print(f"Calculating: {calc}")
    result = monitored_calc._run(calc)
    print(f"Result: {result}\n")

## 9. Best Practices Summary

Key takeaways for building effective agents:

### 🎯 Key Best Practices

1. **Tool Design**
   - Clear, descriptive names and descriptions
   - Comprehensive error handling
   - Input validation and sanitization
   - Appropriate response formats

2. **Security**
   - Validate all inputs
   - Restrict dangerous operations
   - Use safe evaluation methods
   - Implement proper authentication

3. **Performance**
   - Monitor execution time
   - Implement caching where appropriate
   - Handle timeouts gracefully
   - Use async operations for I/O

4. **Reliability**
   - Comprehensive error handling
   - Graceful degradation
   - Retry mechanisms
   - Logging and monitoring

5. **User Experience**
   - Clear error messages
   - Consistent response formats
   - Helpful feedback
   - Progressive disclosure

## 10. Exercise: Build Your Own Agent

Try creating your own custom tool and agent!

In [ ]:
# Exercise: Create a custom tool
# Uncomment and complete the code below

# class YourCustomTool(BaseTool):
#     """Your custom tool description."""
#     name = "YourToolName"
#     description = "Description of what your tool does"
#     
#     def _run(self, input_param: str) -> str:
#         # Implement your tool logic here
#         try:
#             # Your implementation
#             result = f"Processed: {input_param}"
#             return result
#         except Exception as e:
#             return f"Error: {str(e)}"
#     
#     async def _arun(self, input_param: str) -> str:
#         return self._run(input_param)

# Ideas for custom tools:
# - Weather information
# - Unit converter
# - Password generator
# - QR code generator
# - Base64 encoder/decoder
# - Hash calculator

print("💡 Exercise: Create your own custom tool above!")
print("Ideas: Weather checker, unit converter, password generator, etc.")

## Conclusion

This tutorial covered:
- Basic agent setup and tool creation
- Advanced tool development with BaseTool
- Error handling and security considerations
- Performance monitoring and optimization
- Best practices for production agents

### Next Steps
1. Experiment with different agent types
2. Build domain-specific tools
3. Implement advanced memory patterns
4. Explore multi-agent systems
5. Deploy agents in production environments

Happy building! 🚀